# Deribook

#### WIP
ML
matplotlib or other charting library


##### Import, intialize, and set up required dataframes

In [11]:

import asyncio
import websockets
import json
import pandas as pd
import asyncio
import websockets
import json
import pandas as pd
from concurrent.futures import ThreadPoolExecutor
import time
import nest_asyncio
import numpy as np
import matplotlib.pyplot as plt

from datetime import datetime
import tracemalloc
tracemalloc.start()

nest_asyncio.apply()




async def fetch_all_instruments():
    msg = {
        "method": "public/get_instruments",
        "params": {
            "currency": "BTC",
            "expired": "True"
        },
        "jsonrpc": "2.0",
        "id": 1
    }
    async with websockets.connect('wss://www.deribit.com/ws/api/v2') as websocket:
        await websocket.send(json.dumps(msg))
        while websocket.open:
            response = await websocket.recv()
            data = json.loads(response)
            if 'result' in data:
                instruments = data['result']
                return pd.DataFrame(instruments)

async def main():
    all_instruments_df = await fetch_all_instruments()

    # Filter DataFrames based on 'kind'
    kinds = ['future',]  # ['option', 'future', 'spot', 'future_combo', 'option_combo']
    dfs = {kind: all_instruments_df[all_instruments_df['kind'] == kind] for kind in kinds if kind in all_instruments_df['kind'].unique()}

    # Save each DataFrame in the global scope for access in other cells
    for kind, df in dfs.items():
        globals()[f"{kind}_df"] = df
        print(f"{kind.capitalize()} DataFrame created and stored.")

# Run the main function to fetch and categorize instruments
await main()

# Assuming future_df is already defined and populated
# List of columns to keep
columns_to_keep = ['creation_timestamp', 'expiration_timestamp','instrument_name', 'taker_commission', 'maker_commission', 'max_leverage', 'settlement_period']

# Drop all other columns except the ones specified
future_df = future_df[columns_to_keep]


async def fetch_book_summary(instrument_name):
    msg = {
        "jsonrpc": "2.0",
        "id": 1,
        "method": "public/get_book_summary_by_instrument",
        "params": {
            "instrument_name": instrument_name
        }
    }
    url = 'wss://www.deribit.com/ws/api/v2'
    async with websockets.connect(url) as websocket:
        await websocket.send(json.dumps(msg))
        response = await websocket.recv()
        data = json.loads(response)
        if 'result' in data and len(data['result']) > 0:
            result = data['result'][0]
            df = pd.DataFrame([{
                'instrument_name': instrument_name,
                'mid_price': result.get('mid_price'),
                'bid_price': result.get('bid_price'),
                'ask_price': result.get('ask_price'),
                'mark_price': result.get('mark_price'),
                'volume': result.get('volume')
            }])
            return df
        else:
            return pd.DataFrame(columns=['instrument_name', 'mid_price', 'bid_price', 'ask_price', 'mark_price', 'volume'])

# Assuming instr_name is already defined as shown:

# Convert DataFrame column 'instrument_name' directly to a list
instrument_names = future_df['instrument_name'].tolist()

# Fetch data for each instrument name
async def fetch_multiple_instruments(instrument_names):
    tasks = [fetch_book_summary(name) for name in instrument_names]
    dataframes = await asyncio.gather(*tasks)
    combined_df = pd.concat(dataframes, ignore_index=True)
    return combined_df

# Example usage
loop = asyncio.get_event_loop()
result_df = loop.run_until_complete(fetch_multiple_instruments(instrument_names))
pricing_df = pd.merge(future_df, result_df, on='instrument_name')


In [2]:
pricing_df

,creation_timestamp,expiration_timestamp,instrument_name,taker_commission,maker_commission,max_leverage,settlement_period,mid_price,bid_price,ask_price,mark_price,volume
0,1717142422000,1718352000000,BTC-14JUN24,0.0005,-0.0001,25.0,week,69690.00,69687.5,69692.5,69691.22,18.790769
1,1717747214000,1718956800000,BTC-21JUN24,0.0005,-0.0001,25.0,week,69908.75,69907.5,69910.0,69910.42,16.424471
2,1687507209000,1719561600000,BTC-28JUN24,0.0005,0.0000,25.0,month,70052.50,70050.0,70055.0,70050.10,128.885362
3,1717142422000,1721980800000,BTC-26JUL24,0.0005,0.0000,25.0,month,70926.25,70925.0,70927.5,70921.64,25.600109
4,1695974410000,1727424000000,BTC-27SEP24,0.0005,0.0000,25.0,month,72623.75,72622.5,72625.0,72623.61,34.565080
5,1703836813000,1735286400000,BTC-27DEC24,0.0005,0.0000,25.0,month,75072.50,75067.5,75077.5,75079.12,11.337256
6,1711699222000,1743148800000,BTC-28MAR25,0.0005,0.0000,25.0,month,77340.00,77330.0,77350.0,77347.99,2.888134
7,1534242287000,32503708800000,BTC-PERPETUAL,0.0005,0.0000,50.0,perpetual,69583.75,69583.5,69584.0,69584.61,1039.500908


In [3]:
async def fetch_book_summary(instrument_name):
    msg = {
        "jsonrpc": "2.0",
        "id": 1,
        "method": "public/get_book_summary_by_instrument",
        "params": {
            "instrument_name": instrument_name
        }
    }
    url = 'wss://www.deribit.com/ws/api/v2'
    async with websockets.connect(url) as websocket:
        await websocket.send(json.dumps(msg))
        response = await websocket.recv()
        data = json.loads(response)
        if 'result' in data and len(data['result']) > 0:
            result = data['result'][0]
            df = pd.DataFrame([{
                'instrument_name': instrument_name,
                'mid_price': result.get('mid_price'),
                'bid_price': result.get('bid_price'),
                'ask_price': result.get('ask_price'),
                'mark_price': result.get('mark_price'),
                'volume': result.get('volume')
            }])
            return df
        else:
            return pd.DataFrame(columns=['instrument_name', 'mid_price', 'bid_price', 'ask_price', 'mark_price', 'volume'])

# Assuming instr_name is already defined as shown:

# Convert DataFrame column 'instrument_name' directly to a list
instrument_names = future_df['instrument_name'].tolist()

# Fetch data for each instrument name
async def fetch_multiple_instruments(instrument_names):
    tasks = [fetch_book_summary(name) for name in instrument_names]
    dataframes = await asyncio.gather(*tasks)
    combined_df = pd.concat(dataframes, ignore_index=True)
    return combined_df

# Example usage
loop = asyncio.get_event_loop()
result_df = loop.run_until_complete(fetch_multiple_instruments(instrument_names))
print(result_df)

  instrument_name  mid_price  bid_price  ask_price  mark_price       volume
0     BTC-14JUN24   69688.75    69682.5    69695.0    69688.95    18.790769
1     BTC-21JUN24   69907.50    69905.0    69910.0    69908.91    16.424471
2     BTC-28JUN24   70052.50    70042.5    70062.5    70048.80   128.885362
3     BTC-26JUL24   70935.00    70917.5    70952.5    70921.65    25.600109
4     BTC-27SEP24   72618.75    72617.5    72620.0    72622.38    34.565080
5     BTC-27DEC24   75063.75    75057.5    75070.0    75074.90    11.337256
6     BTC-28MAR25   77332.50    77322.5    77342.5    77342.84     2.888134
7   BTC-PERPETUAL   69583.75    69583.5    69584.0    69583.71  1039.500908


In [7]:

# Define the columns
column_labels = ['days to expiry', 't ACT/365', 'mid price', 'bid price', 'ask price', 'mark price', 'spot', 'fair price', 'future price', 
                 '%prem outright', '%prem pa', 'fair roll', 'roll price', '% prem outright', '% prem pa']

# Add these columns to pricing_df with default NaN values
for label in column_labels:
    if label not in pricing_df.columns:
        pricing_df[label] = np.nan


def calculate_metrics(df):
    current_timestamp = datetime.now().timestamp() * 1000  # Convert current time to milliseconds to match the format of expiration_timestamp
    for index, row in df.iterrows():
        if pd.notna(row['expiration_timestamp']):
            # Calculate days to expiry
            time_difference = row['expiration_timestamp'] - current_timestamp
            days_to_expiry = time_difference / (1000 * 60 * 60 * 24)  # Convert milliseconds to days
            df.at[index, 'days to expiry'] = days_to_expiry
        else:
            df.at[index, 'days to expiry'] = np.nan  # Set NaN if expiration_timestamp is not available

# Assuming pricing_df is already defined and contains 'expiration_timestamp'
calculate_metrics(pricing_df)
print(pricing_df[['instrument_name', 'expiration_timestamp', 'days to expiry']])

# Perform calculations
calculate_metrics(pricing_df)

# Display the updated DataFrame
print(pricing_df)

  instrument_name  expiration_timestamp  days to expiry
0     BTC-14JUN24         1718352000000        4.651531
1     BTC-21JUN24         1718956800000       11.651531
2     BTC-28JUN24         1719561600000       18.651531
3     BTC-26JUL24         1721980800000       46.651531
4     BTC-27SEP24         1727424000000      109.651531
5     BTC-27DEC24         1735286400000      200.651531
6     BTC-28MAR25         1743148800000      291.651531
7   BTC-PERPETUAL        32503708800000   356316.651531
   creation_timestamp  expiration_timestamp instrument_name  taker_commission  \
0       1717142422000         1718352000000     BTC-14JUN24            0.0005   
1       1717747214000         1718956800000     BTC-21JUN24            0.0005   
2       1687507209000         1719561600000     BTC-28JUN24            0.0005   
3       1717142422000         1721980800000     BTC-26JUL24            0.0005   
4       1695974410000         1727424000000     BTC-27SEP24            0.0005   
5       17

In [72]:
## Imports

import asyncio
import websockets
import json
import pandas as pd
import asyncio
import websockets
import json
import pandas as pd
from concurrent.futures import ThreadPoolExecutor
import time
import nest_asyncio
import numpy as np
import matplotlib.pyplot as plt

from datetime import datetime
import tracemalloc
tracemalloc.start()

nest_asyncio.apply()



# run to gather all the data

## Fetch Instruments from Deribit

async def fetch_all_instruments():
    msg = {
        "method": "public/get_instruments",
        "params": {
            "currency": "BTC"
        },
        "jsonrpc": "2.0",
        "id": 1
    }
    async with websockets.connect('wss://www.deribit.com/ws/api/v2') as websocket:
        await websocket.send(json.dumps(msg))
        while websocket.open:
            response = await websocket.recv()
            data = json.loads(response)
            if 'result' in data:
                instruments = data['result']
                return pd.DataFrame(instruments)

async def main():
    all_instruments_df = await fetch_all_instruments()

    # Filter DataFrames based on 'kind'
    kinds = ['option', 'future', 'spot', 'future_combo', 'option_combo']
    dfs = {kind: all_instruments_df[all_instruments_df['kind'] == kind] for kind in kinds if kind in all_instruments_df['kind'].unique()}

    # Save each DataFrame in the global scope for access in other cells
    for kind, df in dfs.items():
        globals()[f"{kind}_df"] = df
        print(f"{kind.capitalize()} DataFrame created and stored.")

# Run the main function to fetch and categorize instruments
await main()


creation_expiration = future_df[['instrument_name', 'expiration_timestamp', 'creation_timestamp', 'settlement_period']]




async def fetch_book_summary(instrument_name, retries=5, delay=1):
    msg = {
        "jsonrpc": "2.0",
        "id": 1,
        "method": "public/get_book_summary_by_instrument",
        "params": {
            "instrument_name": instrument_name
        }
    }
    url = 'wss://www.deribit.com/ws/api/v2'
    for attempt in range(retries):
        try:
            async with websockets.connect(url) as websocket:
                await websocket.send(json.dumps(msg))
                response = await websocket.recv()
                data = json.loads(response)
                if 'result' in data and len(data['result']) > 0:
                    result = data['result'][0]
                    return instrument_name, {
                        'mid_price': result.get('mid_price'),
                        'bid_price': result.get('bid_price'),
                        'ask_price': result.get('ask_price'),
                        'mark_price': result.get('mark_price'),
                        'volume': result.get('volume')
                    }
        except websockets.exceptions.InvalidStatusCode as e:
            if e.status_code == 429:
                await asyncio.sleep(delay * (2 ** attempt))  # Exponential backoff
            else:
                raise  # Re-raise exceptions that aren't rate limits
        except Exception as e:
            print(f"Failed to fetch data for {instrument_name}: {str(e)}")
            await asyncio.sleep(delay)  # Wait before retrying for other errors
    return instrument_name, {}  # Return empty result after all retries fail

# Adjust the main function and other parts of your code to use this updated fetch function.

async def fetch_all_book_summaries(df):
    instrument_names = df['instrument_name'].tolist()
    tasks = [fetch_book_summary(name) for name in instrument_names]
    results = {}
    for future in asyncio.as_completed(tasks):
        name, summary = await future
        results[name] = summary
    return results

async def calculate_days_to_expiry(row):
    expiration_date = datetime.fromtimestamp(row['expiration_timestamp'] / 1000)
    current_date = datetime.now()
    days_to_expiry = (expiration_date - current_date).days
    return days_to_expiry

async def main():
    # Fetch summaries for future DataFrame only
    future_book_summaries = await fetch_all_book_summaries(future_df)
    future_book_df = pd.DataFrame(future_book_summaries).T
   

    # Define the columns
    column_labels = ['contract value', 'days to expiry', 't ACT/365', 'mark price', 'spot', 'fair price', 'future price', 
                     '%prem outright', '%prem pa', 'fair roll', 'roll price', '% prem outright', '% prem pa']

    # Reindex the DataFrame to include all specified columns, filling missing columns with NaN
    pricing_df = future_book_df.reindex(columns=column_labels)

    # Update pricing_df with the price and volume data from future_book_df
    if all(col in future_book_df.columns for col in ['mid_price', 'bid_price', 'ask_price', 'mark_price', 'volume']):
        pricing_df.update(future_book_df[['mid_price', 'bid_price', 'ask_price', 'mark_price', 'volume']])

    # Calculate days to expiry for instruments in future_df that are also in pricing_df
    current_timestamp = datetime.now().timestamp()
    future_df['days to expiry'] = future_df.apply(lambda row: (datetime.fromtimestamp(row['expiration_timestamp'] / 1000) - datetime.fromtimestamp(current_timestamp)).days if row['instrument_name'] in pricing_df.index else np.nan, axis=1)

    # Map the 'days to expiry' from future_df to pricing_df
    pricing_df['days to expiry'] = pricing_df.index.map(future_df.set_index('instrument_name')['days to expiry'])

    # Calculate 't ACT/365'
    pricing_df['t ACT/365'] = pricing_df['days to expiry'] / 365

    # Print the DataFrame to verify
    print(pricing_df)

await main()


Option DataFrame created and stored.
Future DataFrame created and stored.
Spot DataFrame created and stored.
Future_combo DataFrame created and stored.
Option_combo DataFrame created and stored.
               contract value  days to expiry   t ACT/365  mid price  \
BTC-21JUN24               NaN              13    0.035616        NaN   
BTC-28MAR25               NaN             293    0.802740        NaN   
BTC-26JUL24               NaN              48    0.131507        NaN   
BTC-27SEP24               NaN             111    0.304110        NaN   
BTC-PERPETUAL             NaN          356318  976.213699        NaN   
BTC-27DEC24               NaN             202    0.553425        NaN   
BTC-28JUN24               NaN              20    0.054795        NaN   
BTC-14JUN24               NaN               6    0.016438        NaN   

               bid price  ask price  mark price  spot  fair price  \
BTC-21JUN24          NaN        NaN         NaN   NaN         NaN   
BTC-28MAR25       

##### Quanto Futures

In [25]:
# futures purps

contracts = 10000
multiplier = 0.001

NameError: name 'spot_book_df' is not defined

###### Quickstyles version

In [10]:
def calculate_pnl(position_type, position_dollar, entry_price, exit_price):
    """
    Calculate PnL for a trading position.

    Args:
    position_type (str): "Short" or other (assuming "Long")
    position_dollar (float): The dollar amount of the position
    entry_price (float): The price at which the position was entered
    exit_price (float): The price at which the position was exited

    Returns:
    tuple: Returns a tuple containing PnL in BTC and PnL in USD
    """
    if position_type == "Short":
        pnl_btc = ((entry_price - exit_price) / exit_price) * (position_dollar / entry_price)
        pnl_usd = (entry_price - exit_price) * (position_dollar / entry_price)
    else:
        pnl_btc = ((entry_price - exit_price) / exit_price) * (position_dollar / entry_price) * -1
        pnl_usd = (entry_price - exit_price) * (position_dollar / entry_price) * -1

    return pnl_btc, pnl_usd

# Example usage:
position_type = "Long"
position_dollar = 3850
entry_price = 3850
exit_price = 4000

pnl_btc, pnl_usd = calculate_pnl(position_type, position_dollar, entry_price, exit_price)
print(f"PNL (BTC): {pnl_btc}")
print(f"PNL ($): {pnl_usd}")

PNL (BTC): 0.0375
PNL ($): 150.0


In [75]:


async def fetch_book_summary(instrument_name, retries=5, delay=1):
    msg = {
        "jsonrpc": "2.0",
        "id": 1,
        "method": "public/get_book_summary_by_instrument",
        "params": {
            "instrument_name": instrument_name
        }
    }
    url = 'wss://www.deribit.com/ws/api/v2'
    for attempt in range(retries):
        try:
            async with websockets.connect(url) as websocket:
                await websocket.send(json.dumps(msg))
                response = await websocket.recv()
                data = json.loads(response)
                if 'result' in data and len(data['result']) > 0:
                    result = data['result'][0]
                    return instrument_name, {
                        'mid_price': result.get('mid_price'),
                        'bid_price': result.get('bid_price'),
                        'ask_price': result.get('ask_price'),
                        'mark_price': result.get('mark_price'),
                        'volume': result.get('volume')
                    }
        except websockets.exceptions.InvalidStatusCode as e:
            if e.status_code == 429:
                await asyncio.sleep(delay * (2 ** attempt))  # Exponential backoff
            else:
                raise  # Re-raise exceptions that aren't rate limits
        except Exception as e:
            print(f"Failed to fetch data for {instrument_name}: {str(e)}")
            await asyncio.sleep(delay)  # Wait before retrying for other errors
    return instrument_name, {}  # Return empty result after all retries fail

# Adjust the main function and other parts of your code to use this updated fetch function.

async def fetch_all_book_summaries(df):
    instrument_names = df['instrument_name'].tolist()
    tasks = [fetch_book_summary(name) for name in instrument_names]
    results = {}
    for future in asyncio.as_completed(tasks):
        name, summary = await future
        results[name] = summary
    return results

async def main():
    # Fetch summaries for each DataFrame
    spot_book_summaries = await fetch_all_book_summaries(spot_df)
    spot_book_df = pd.DataFrame(spot_book_summaries).T  # Transpose to match your desired format

    future_book_summaries = await fetch_all_book_summaries(future_df)
    future_book_df = pd.DataFrame(future_book_summaries).T

    future_combo_book_summaries = await fetch_all_book_summaries(future_combo_df)
    future_combo_book_df = pd.DataFrame(future_combo_book_summaries).T

    option_combo_book_summaries = await fetch_all_book_summaries(option_combo_df)
    option_combo_book_df = pd.DataFrame(option_combo_book_summaries).T


    # Assuming spot_book_df and future_book_df are already defined and populated
    # Concatenate both DataFrames
    combined_df = pd.concat([spot_book_df, future_book_df])

    # Set 'instrument_name' as the index if it's not already
    combined_df.index.name = 'instrument_name'

    # Define the columns
    column_labels = ['contract value', 'days to expiry', 't ACT/365', 'spot', 'fair price', 'future price', 
                    '%prem outright', '%prem pa', 'fair roll', 'roll price', '% prem outright', '% prem pa']

    # Reindex the DataFrame to include all specified columns, filling missing columns with NaN
    pricing_df = combined_df.reindex(columns=column_labels)

    # Print the DataFrame to verify
    print(pricing_df)

await main()

                 contract value  days to expiry  t ACT/365  spot  fair price  \
instrument_name                                                                
ETH_BTC                     NaN             NaN        NaN   NaN         NaN   
BTC_USDC                    NaN             NaN        NaN   NaN         NaN   
BTC_USDT                    NaN             NaN        NaN   NaN         NaN   
BTC-27DEC24                 NaN             NaN        NaN   NaN         NaN   
BTC-28JUN24                 NaN             NaN        NaN   NaN         NaN   
BTC-21JUN24                 NaN             NaN        NaN   NaN         NaN   
BTC-28MAR25                 NaN             NaN        NaN   NaN         NaN   
BTC-14JUN24                 NaN             NaN        NaN   NaN         NaN   
BTC-26JUL24                 NaN             NaN        NaN   NaN         NaN   
BTC-27SEP24                 NaN             NaN        NaN   NaN         NaN   
BTC-PERPETUAL               NaN         

In [18]:
future_book_df

NameError: name 'future_book_df' is not defined

In [5]:
future_df

,tick_size_steps,quote_currency,min_trade_amount,counter_currency,settlement_period,settlement_currency,block_trade_tick_size,block_trade_min_trade_amount,block_trade_commission,option_type,...,instrument_type,instrument_name,taker_commission,maker_commission,kind,rfq,price_index,max_liquidation_commission,max_leverage,future_type
183,[],USD,10.0,USD,week,BTC,0.01,200000.0,0.00025,NaN,...,reversed,BTC-7JUN24,0.0005,-0.0001,future,False,btc_usd,0.0075,25.0,reversed
258,[],USD,10.0,USD,week,BTC,0.01,200000.0,0.00025,NaN,...,reversed,BTC-14JUN24,0.0005,-0.0001,future,False,btc_usd,0.0075,25.0,reversed
384,[],USD,10.0,USD,month,BTC,0.01,200000.0,0.00025,NaN,...,reversed,BTC-28JUN24,0.0005,0.0000,future,False,btc_usd,0.0075,25.0,reversed
522,[],USD,10.0,USD,month,BTC,0.01,200000.0,0.00025,NaN,...,reversed,BTC-26JUL24,0.0005,0.0000,future,False,btc_usd,0.0075,25.0,reversed
708,[],USD,10.0,USD,month,BTC,0.01,200000.0,0.00025,NaN,...,reversed,BTC-27SEP24,0.0005,0.0000,future,False,btc_usd,0.0075,25.0,reversed
842,[],USD,10.0,USD,month,BTC,0.01,200000.0,0.00025,NaN,...,reversed,BTC-27DEC24,0.0005,0.0000,future,False,btc_usd,0.0075,25.0,reversed
969,[],USD,10.0,USD,month,BTC,0.01,200000.0,0.00025,NaN,...,reversed,BTC-28MAR25,0.0005,0.0000,future,False,btc_usd,0.0075,25.0,reversed
1019,[],USD,10.0,USD,perpetual,BTC,0.01,200000.0,0.00025,NaN,...,reversed,BTC-PERPETUAL,0.0005,0.0000,future,False,btc_usd,0.0075,50.0,reversed


In [9]:
# Spot Trading

spot_equity = 1000
spot_xbtusd = 100
spot_buying_power = spot_equity / spot_xbtusd

spot_price_change = 0.1
spot_pnl = spot_xbtusd * spot_price_change * spot_buying_power
spot_roi = spot_pnl / spot_equity

spot_buying_power

10.0

In [ ]:
# Margin Trading
margin_init_margin = 0.3
margin_equity = 100

margin_total_usd = spot_equity / margin_init_margin
margin_borrowed_funds = margin_total_usd - margin_equity
margin_buying_power = margin_total_usd / spot_xbtusd
margin_price_change = 0.1
margin_pnl = spot_xbtusd * margin_price_change * margin_buying_power
margin_roi = margin_pnl / margin_equity

daily_interest_rate = 0.001
daily_interest_cost = daily_interest_rate * margin_borrowed_funds



In [ ]:
# Futures Trading
future_equity = 1000
spot_xbtusd = 100
multiplier = 0.01
contract_value = 10
initial_margin = 
future_buying_power = (future_equity / initial_margin) / contract_value




future_price_change = 0.1
future_pnl = future_price_change * spot_xbtusd * multiplier * future_buying_power
future_roi = future_pnl / future_equity

future_pnl
future_roi


Types of Bitcoin Futures:

Quanto Futures - Fixed Bitcoin multiplier applied to the USD price
XBT Value = Price * Multiplier * Contracts
USD Value = Price^2 * Multiplier * Contracts
Linear in Bitcoin terms, quadratic in USD terms


In [5]:
# types of bitcoin futures





Cash and Carry

In [4]:
import pandas as pd
from datetime import datetime

# Assuming future_df has necessary columns like 'expiry_date', 'future_price', etc.
# Calculate days to expiry assuming 'expiry_date' is in YYYY-MM-DD format
future_df['days_to_expiry'] = (pd.to_datetime(future_df['expiry_date']) - datetime.now()).dt.days
future_df['t ACT/365'] = future_df['days_to_expiry'] / 365

# Define the columns
column_labels = ['contract value', 'days to expiry', 't ACT/365', 'spot', 'fair price', 'future price', 
                 '%prem outright', '%prem pa', 'fair roll', 'roll price', '% prem outright', '% prem pa']

# Create the DataFrame with instrument_name as the index
pricing_df = pd.DataFrame(index=future_df['instrument_name'], columns=column_labels)

# Fill the DataFrame with calculated values or default values
for index, row in future_df.iterrows():
    instrument_name = row['instrument_name']
    pricing_df.loc[instrument_name, 'days to expiry'] = row['days_to_expiry']
    pricing_df.loc[instrument_name, 't ACT/365'] = row['t ACT/365']
    pricing_df.loc[instrument_name, 'future price'] = row['future_price']
    # Add more calculations as needed
    # Example:
    # pricing_df.loc[instrument_name, 'contract value'] = calculate_contract_value(row)
    # pricing_df.loc[instrument_name, 'fair price'] = calculate_fair_price(row)

# Display the DataFrame
print(pricing_df)

                BTC-7JUN24 BTC-14JUN24 BTC-28JUN24 BTC-26JUL24 BTC-27SEP24  \
contract value         NaN         NaN         NaN         NaN         NaN   
days to expiry         NaN         NaN         NaN         NaN         NaN   
t ACT/365              NaN         NaN         NaN         NaN         NaN   
spot                   NaN         NaN         NaN         NaN         NaN   
fair price             NaN         NaN         NaN         NaN         NaN   
future price           NaN         NaN         NaN         NaN         NaN   
%prem outright         NaN         NaN         NaN         NaN         NaN   
%prem pa               NaN         NaN         NaN         NaN         NaN   
fair roll              NaN         NaN         NaN         NaN         NaN   
roll price             NaN         NaN         NaN         NaN         NaN   
% prem outright        NaN         NaN         NaN         NaN         NaN   
% prem pa              NaN         NaN         NaN         NaN  

In [5]:
spot_df

,tick_size_steps,quote_currency,min_trade_amount,counter_currency,settlement_period,settlement_currency,block_trade_tick_size,block_trade_min_trade_amount,block_trade_commission,option_type,...,instrument_type,instrument_name,taker_commission,maker_commission,kind,rfq,price_index,max_liquidation_commission,max_leverage,future_type
1020,[],USDC,0.0001,USDC,NaN,NaN,0.0001,10.0,0.0,NaN,...,linear,BTC_USDC,0.0,0.0,spot,False,btc_usdc,NaN,NaN,NaN
1021,[],USDT,0.0001,USDT,NaN,NaN,0.0001,10.0,0.0,NaN,...,linear,BTC_USDT,0.0,0.0,spot,False,btc_usdt,NaN,NaN,NaN
1022,[],BTC,0.0001,BTC,NaN,NaN,0.0001,10.0,0.0,NaN,...,linear,ETH_BTC,0.0,0.0,spot,False,eth_btc,NaN,NaN,NaN


In [8]:


def update_instrument_data_df(spot_df, future_df):
    # Calculate the instrument names
    instrument_names = future_df['instrument_name'].tolist()

    # Define the index labels
    index_labels = ['contract value', 'expiry date', 't ACT/365', 'spot', 'fair price', 'future price', 
                    '%prem outright', '%prem pa', 'fair roll', 'roll price', '% prem outright', '% prem pa']

    # Create an empty DataFrame with these indices and columns
    instrument_data_df = pd.DataFrame(index=index_labels, columns=instrument_names)

    # Assuming necessary columns are present in the DataFrames
    # Calculate days to expiry and other metrics
    future_df['days_to_expiry'] = (pd.to_datetime(future_df['expiry_date']) - datetime.now()).dt.days
    future_df['t ACT/365'] = future_df['days_to_expiry'] / 365

    # Assuming spot_df has a 'spot_price' column
    spot_prices = spot_df.set_index('instrument_name')['spot_price']

    # Fill the DataFrame with calculated values
    for instrument in instrument_names:
        instrument_data_df.loc['expiry date', instrument] = future_df.loc[future_df['instrument_name'] == instrument, 'expiry_date'].values[0]
        instrument_data_df.loc['t ACT/365', instrument] = future_df.loc[future_df['instrument_name'] == instrument, 't ACT/365'].values[0]
        instrument_data_df.loc['spot', instrument] = spot_prices.get(instrument, pd.NA)
        instrument_data_df.loc['future price', instrument] = future_df.loc[future_df['instrument_name'] == instrument, 'future_price'].values[0]
        # Additional calculations for 'contract value', 'fair price', etc.
        # Example:
        # instrument_data_df.loc['contract value', instrument] = calculate_contract_value(instrument)
        # instrument_data_df.loc['fair price', instrument] = calculate_fair_price(instrument)

    # Return the updated DataFrame
    return instrument_data_df

# Example usage
instrument_data_df = update_instrument_data_df(spot_df, future_df)
print(instrument_data_df)

KeyError: 'expiry_date'